In [ ]:
using DifferentialEquations
using Plots; pyplot()

In [ ]:
function droplet_dae(out, du, u, p, t)
    α, β, θ₀, a, b, ρm, ρb, hm = p
    rb, hb = u
    drb, dhb = du
    
    rm = α*t^β
    drm = α*β*t^(β-1)
    
    θ = θ₀ + a*t^(-b)
    
    # Conservation of mass
    out[1] = ρb/(ρm*hm) * (3*rb*drb + hb*dhb) + 6*rm*drm
    # Relate θ to rb, hb
    out[2] = sin(θ) - 2*rb*hb/(rb^2+hb^2)
end

In [ ]:
function droplet_ode(du, u, p, t)
    α, β, θ₀, a, b, ρm, ρb, hm = p
    #rb, hb = u
    rb = u[1]
    hb = u[2]
    
    rm = α*t^β
    drm = α*β*t^(β-1)
    
    θ = θ₀ + a*t^(-b)
    dθ = -a*b*t^(-b-1)
    
    σ = ρb/(ρm*hm)
    
    A = [3*σ*rb σ*hb; hb -rb]
    b = [-6*rm*drm; cos(θ)*dθ/2 * (hb^2+rb^2)^2/(hb^2-rb^2)]
    
    du1 = A\b
    du[1] = du1[1]
    du[2] = du1[2]
    
    #println("A=$A")
    #println("b=$b")
    #println("du = $du")
end

In [ ]:
# Define parameters
α = 104.
β = 0.2
θ₀ = 0.29
a = 0.27
b = .59

ρm = 3.
ρb = 1.
hm = 1.5
p = (α, β, θ₀, a, b, ρm, ρb, hm)

# Initial conditions
rb₀ = 50.
hb₀ = 30.

R₀ = (rb₀^2+hb₀^2)/(2*hb₀)
θi = acos(1-hb₀/R₀)
println("R₀ = $R₀")
println("θi = $(θi*180/pi)")

u₀ = [rb₀; hb₀]
du₀ = [1.; -1.]

# Solver variables
t₀ = ((θi-θ₀)/a)^(-1/b)+.1 # Start at time that θ matches rb₀ and hb₀.
println("t₀ = $t₀")
tmax = 5
tspan = (t₀, tmax)

In [ ]:
function plot_droplet(rb, rm, hb, hm)
    R = (rb^2+hb^2)/(2*hb)
    θ = acos(1-hb/R)
    
    # Plot circle
    th_cir = linspace(pi/2-θ,pi/2+θ,51)
    x_cir = R*cos.(th_cir)
    y_cir = R*sin.(th_cir)
    plot(x_cir, y_cir)
    
    # Plot monolayer
    x_ml = [-mr, mr, mr, -mr]
    y_ml = [0, 0, hm, hm]
    plot!(x_ml, y_ml)
end

In [ ]:
plot_droplet(50, 50, 20, 2)

In [ ]:
ode_prob = ODEProblem(droplet_ode, u₀, tspan, p)
sol = solve(ode_prob, Rosenbrock23(), abstol=1e-8, reltol=1e-8, dtmin=1e-12)

In [ ]:
plot(sol)
θ = θ₀+a*sol.t.^(-b)
plot!(sol.t, θ*180/pi)
rm = α*sol.t.^β
plot!(sol.t, rm)
ylims!(-10, 100)

In [ ]:
# Solve DAE
differential_vars = [true, true];
dae_prob = DAEProblem(droplet_dae, du₀, u₀, tspan, p, differential_vars=differential_vars)
sol = solve(dae_prob, IDA())